In [64]:
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from scipy.stats import binom
import datetime



In [66]:
# Define stock and date range
ticker = "HDFCBANK.NS"
start = datetime.datetime(2024, 4, 1)
end = datetime.datetime(2025, 4, 30)

# Downloading daily stock data
stock_df = yf.download(ticker, start=start, end=end, interval="1d", group_by='ticker')
stock_df = stock_df.dropna()
stock_df.index = pd.to_datetime(stock_df.index)

[*********************100%***********************]  1 of 1 completed


In [67]:
fig = go.Figure(data=[
    go.Candlestick(
        x=stock_df.index,
        open=stock_df[ticker]['Open'],
        high=stock_df[ticker]['High'],
        low=stock_df[ticker]['Low'],
        close=stock_df[ticker]['Close']
    ),
    go.Scatter(
        x=['2024-06-05'], y=[3881],
        mode='markers+text',
        marker=dict(size=12, color='red'),
        text=["Major Drop"], textposition="top center",
        name="Notable Point"
    ),
    go.Scatter(
        x=['2025-03-02', '2025-03-02'],
        y=[stock_df[ticker]['Low'].min() - 10, stock_df[ticker]['High'].max() + 10],
        mode='lines',
        line=dict(color='green', width=4, dash='dot'),
        name='Key Trend Marker'
    )
])

fig.update_layout(
    title=f"{ticker} - Price Action Overview",
    xaxis_title='Date',
    yaxis_title='Stock Price (INR)',
    template='plotly_dark',
    hovermode='x unified',
    xaxis_rangeslider_visible=False
)
fig.show()

In [68]:
# Adding return columns
stock_df[(ticker, 'Daily Return')] = stock_df[ticker]['Close'].pct_change()
stock_df[(ticker, 'Log Return')] = np.log(stock_df[ticker]['Close'] / stock_df[ticker]['Close'].shift(1))

# Visualization
fig = make_subplots(rows=1, cols=2, subplot_titles=["Simple Return", "Logarithmic Return"])

fig.add_trace(go.Scatter(x=stock_df.index, y=stock_df[(ticker, 'Daily Return')], name="Simple"), row=1, col=1)
fig.add_trace(go.Scatter(x=stock_df.index, y=stock_df[(ticker, 'Log Return')], name="Log"), row=1, col=2)

fig.update_layout(title_text="Daily Returns Comparison", template="plotly_dark", title_x=0.5)
fig.show()


In [69]:
# 14-day rolling standard deviation
stock_df.loc[:, (ticker, 'Rolling Volatility')] = stock_df[ticker]['Daily Return'].rolling(window=14).std()

fig = px.line(stock_df, x=stock_df.index, y=stock_df[ticker]['Rolling Volatility'],
              title='14-Day Rolling Volatility')
fig.update_layout(xaxis_title='Date', yaxis_title='Volatility')
fig.show()


**Comparison Between Simple and Logarithmic Returns**

**Simple returns** represent the percentage change in an asset’s price and are calculated as:
    **Rₜ = (Pₜ - Pₜ₋₁) / Pₜ₋₁**

**Log returns** (also known as continuously compounded returns) are computed using natural logarithms:
    **Rₜ = ln(Pₜ / Pₜ₋₁)**

For daily stock data, where price fluctuations tend to be small, both types of returns appear nearly identical. This is because when the change is minimal, the logarithmic approximation **ln(1 + x) ≈ x** holds true. However, when prices shift dramatically—such as during significant news events or market crashes—log returns may diverge more noticeably from simple returns due to their compounding characteristics.

A key advantage of log returns is their **additive nature over time**, which means returns across multiple periods can be simply summed. This makes them especially useful in financial modeling and portfolio optimization.

On the other hand, **simple returns** are more intuitive and are typically used in performance summaries, as they directly show the gain or loss as a percentage.
In summary, both types have their place: **log returns** are preferred for mathematical and modeling purposes, while **simple returns** are favored for reporting and communication.


Problem Statement - 3

In [70]:
# Labeling daily movement
stock_df.loc[:, (ticker, 'Market Move')] = stock_df[ticker]['Daily Return'].apply(lambda x: 'UP' if x > 0 else 'DOWN')

# Compute probabilities
total_days = stock_df[ticker]['Market Move'].count()
up_days = (stock_df[ticker]['Market Move'] == 'UP').sum()
p_up = up_days / total_days
p_down = 1 - p_up

print(f"Empirical Probability of UP day: {p_up:.3f}")


Empirical Probability of UP day: 0.541


In [71]:
# Binomial probabilities
theo_p_6_up = binom.pmf(k=6, n=10, p=0.6)
theo_p_8plus_up = 1 - binom.cdf(k=7, n=10, p=0.6)

emp_p_6_up = binom.pmf(k=6, n=10, p=p_up)
emp_p_8plus_up = 1 - binom.cdf(k=7, n=10, p=p_up)

print(f"Theoretical P(exactly 6 UP days in 10): {theo_p_6_up:.3f}")
print(f"Theoretical P(8+ UP days in 10): {theo_p_8plus_up:.3f}")
print(f"Empirical P(exactly 6 UP days): {emp_p_6_up:.3f}")
print(f"Empirical P(8+ UP days): {emp_p_8plus_up:.3f}")



Theoretical P(exactly 6 UP days in 10): 0.251
Theoretical P(8+ UP days in 10): 0.167
Empirical P(exactly 6 UP days): 0.234
Empirical P(8+ UP days): 0.090


In [72]:
simulated_results = np.random.binomial(n=10, p=0.6, size=1000)
sim_6 = np.mean(simulated_results == 6)
sim_8plus = np.mean(simulated_results >= 8)

print(f"Simulated P(6 heads out of 10): {sim_6:.3f}")
print(f"Simulated P(8+ heads out of 10): {sim_8plus:.3f}")


Simulated P(6 heads out of 10): 0.259
Simulated P(8+ heads out of 10): 0.155


In [73]:
up_mean = stock_df[ticker].loc[stock_df[ticker]['Market Move'] == 'UP', 'Daily Return'].mean()
down_mean = stock_df[ticker].loc[stock_df[ticker]['Market Move'] == 'DOWN', 'Daily Return'].mean()

print(f"Avg return on UP days: {up_mean:.4f}")
print(f"Avg return on DOWN days: {down_mean:.4f}")



Avg return on UP days: 0.0101
Avg return on DOWN days: -0.0095


In [74]:
results = []
data = stock_df[ticker][['Daily Return', 'Market Move']].copy()

for i in range(len(data) - 9):
    window = data.iloc[i:i+10]
    up_days = window[window['Market Move'] == 'UP']['Daily Return']
    down_days = window[window['Market Move'] == 'DOWN']['Daily Return']

    if len(up_days) >= 6:
        if down_days.mean() < up_days.mean():
            results.append((data.index[i], data.index[i+9]))

for start, end in results:
    print(f"Window with >=60% UP but net loss: {start.date()} to {end.date()}")


Window with >=60% UP but net loss: 2024-04-02 to 2024-04-16
Window with >=60% UP but net loss: 2024-05-06 to 2024-05-17
Window with >=60% UP but net loss: 2024-05-08 to 2024-05-22
Window with >=60% UP but net loss: 2024-05-09 to 2024-05-23
Window with >=60% UP but net loss: 2024-05-10 to 2024-05-24
Window with >=60% UP but net loss: 2024-05-13 to 2024-05-27
Window with >=60% UP but net loss: 2024-05-14 to 2024-05-28
Window with >=60% UP but net loss: 2024-05-15 to 2024-05-29
Window with >=60% UP but net loss: 2024-05-16 to 2024-05-30
Window with >=60% UP but net loss: 2024-05-17 to 2024-05-31
Window with >=60% UP but net loss: 2024-05-21 to 2024-06-03
Window with >=60% UP but net loss: 2024-05-22 to 2024-06-04
Window with >=60% UP but net loss: 2024-05-23 to 2024-06-05
Window with >=60% UP but net loss: 2024-05-24 to 2024-06-06
Window with >=60% UP but net loss: 2024-05-27 to 2024-06-07
Window with >=60% UP but net loss: 2024-05-28 to 2024-06-10
Window with >=60% UP but net loss: 2024-

In [75]:
expected_return = 250 * p_up - 150
print(f"Expected Profit per Trade Day: ${expected_return:.2f}")


Expected Profit per Trade Day: $-14.66


**Part 3: Evaluating Bet Profitability**

Let’s analyze whether this betting strategy is profitable. The overall return depends on the earnings from the days the stock rises versus the losses from the days it falls.

The net return can be expressed using the formula:
**Net Return = (Earnings per UP day × Probability of UP) − (Loss per DOWN day × Probability of DOWN)**

According to the conditions of the bet:

* You gain \$100 on a day when the stock price increases.
* You lose \$150 on a day when the stock price decreases.
* Since there are only two outcomes (UP or DOWN), we can say **P(DOWN) = 1 − P(UP)**.

Substituting these values into the formula gives:
**Net Return = (100 × P(UP)) − (150 × (1 − P(UP)))**
Simplifying further:
**Net Return = 100 × P(UP) − 150 + 150 × P(UP)**
**Net Return = 250 × P(UP) − 150**

To make a profit, this net return must be greater than zero:
**250 × P(UP) − 150 > 0**
**250 × P(UP) > 150**
**P(UP) > 150 / 250**
**P(UP) > 0.6**

**Conclusion:**
For this bet to yield a positive return, the probability of the stock going up on any given day must be greater than **60%**.
